In [1]:
import pandas as pd
import pickle
import networkx as nx
from tqdm import tqdm

In [2]:
domens = ['science', 'environment']
all_nodes = []
for domen in domens:
    path = '../TExEval-2_testdata_1.2/gs_taxo/EN/' + domen + '_eurovoc_en.taxo'
    G = nx.DiGraph()

    with open(path, 'r') as f:
        for line in f:
            idx, hypo, hyper = line.split('\t')
            hyper = hyper.replace('\n', '')
            G.add_node(hypo)
            G.add_node(hyper)
            G.add_edge(hyper, hypo)

    all_nodes.extend(list(G.nodes))

path = '../TExEval-2_testdata_1.2/gs_taxo/EN/' + 'food' + '_wordnet_en.taxo'
G = nx.DiGraph()

with open(path, 'r') as f:
    for line in f:
        idx, hypo, hyper = line.split('\t')
        hyper = hyper.replace('\n', '')
        G.add_node(hypo)
        G.add_node(hyper)
        G.add_edge(hyper, hypo)

all_nodes.extend(list(G.nodes))

In [3]:
train_path = '../babel_datasets/v2_wnet_lemmas_train.pickle'
with open(train_path, 'rb') as f:
    train = pickle.load(f)

In [4]:
def clean(word):
    return word.split('.')[0]

def check_is_inside(word, elem):
    all_words = []
    levels = ['children', 'parents', 'grandparents']
    for level in levels:
        if elem[level] is not None:
            if isinstance(elem[level], list):
                for c in elem[level]:
                    all_words.append(clean(c))
            else:
                all_words.append(clean(elem[level]))
    
    if 'brothers' in elem.keys():
        for c in elem['brothers']:
            all_words.append(clean(c))

   # print(all_words)
    return word in all_words
    
def check_if_target(targets, elem):
    for target in targets:
        if check_is_inside(target, elem):
            return True
    
    return False

In [5]:
new_train = []
for elem in tqdm(train):
    if check_if_target(all_nodes, elem):
        #print(elem)
        continue
    new_train.append(elem)

  0%|          | 0/20236 [00:00<?, ?it/s]

100%|██████████| 20236/20236 [01:09<00:00, 290.86it/s]


In [11]:
new_path = '../babel_datasets/v2_wnet_lemmas_train_no_semeval16.pickle'
with open(new_path, 'wb') as f:
    pickle.dump(new_train, f)